1 - bibliotecas

In [ ]:
import arxiv
import requests
import os
import re
import json
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import torch
import faiss
import pickle
import numpy as np
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from sklearn.metrics.pairwise import cosine_similarity

2 - download dos artigos

In [ ]:
os.makedirs("pdfs", exist_ok=True)

# 2. Configura o cliente e a busca
client = arxiv.Client()
search = arxiv.Search(
    query="cat:astro-ph",
    max_results=1000,
    sort_by=arxiv.SortCriterion.Relevance
)

print("Iniciando busca e download...")

for i, result in enumerate(client.results(search)):
    pdf_url = result.pdf_url
    safe_title = "".join(c for c in result.title if c.isalnum() or c in (' ', '_')).rstrip()
    filename = f"pdfs/paper_{i}_{safe_title[:50]}.pdf"
    
    print(f"Baixando: {result.title}...")
    
    try:
        r = requests.get(pdf_url)
        r.raise_for_status() 
        
        with open(filename, "wb") as f:
            f.write(r.content)
            
    except Exception as e:
        print(f"Erro ao baixar o artigo {i}: {e}")

print("\nProcesso finalizado!")

3 - extração de textos dos PDFs

In [ ]:
path = "pdfs"
output_dir = "processed_data"
os.makedirs(output_dir, exist_ok=True)

texts_metadata = []

def clean_academic_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'http\S+', '', text)
    return text.strip()

if os.path.exists(path):
    files = sorted([f for f in os.listdir(path) if f.endswith(".pdf")])
    print(f"Iniciando a leitura de {len(files)} arquivos...")

    for i, file in enumerate(files):
        try:
            reader = PdfReader(os.path.join(path, file))
            full_text = ""
            
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    full_text += page_text + " "
            
            cleaned_text = clean_academic_text(full_text)
            
            if len(cleaned_text) > 100:
                
                texts_metadata.append({
                    "source": file,
                    "text": cleaned_text
                })
            
            if (i + 1) % 50 == 0:
                print(f"Progresso: {i + 1}/{len(files)} arquivos processados.")
            
        except Exception as e:
            print(f"Erro no arquivo {file}: {e}")

    with open(os.path.join(output_dir, "extracted_texts.json"), "w", encoding="utf-8") as f:
        json.dump(texts_metadata, f, ensure_ascii=False, indent=4)
    
    print(f"\nExtração concluída! {len(texts_metadata)} textos salvos em '{output_dir}/extracted_texts.json'")

else:
    print("A pasta 'pdfs' não foi encontrada.")

4 - chunckerização

In [ ]:
input_dir = "processed_data"
output_dir = "processed_data"
json_input = os.path.join(input_dir, "extracted_texts.json")

if os.path.exists(json_input):
    with open(json_input, "r", encoding="utf-8") as f:
        texts_metadata = json.load(f)
else:
    print("Erro: extracted_texts.json não encontrado. Rode o código de extração primeiro.")
    texts_metadata = []

def chunk_text_with_overlap(text, size=500, overlap=50):
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), size - overlap):
        chunk = " ".join(words[i : i + size])
        chunks.append(chunk)
        if i + size >= len(words):
            break
            
    return chunks

all_chunks_with_metadata = []

print(f"Iniciando a divisão de {len(texts_metadata)} documentos...")

for entry in texts_metadata:
    source_name = entry["source"]
    text_content = entry["text"]
    
    doc_chunks = chunk_text_with_overlap(text_content, size=512, overlap=50)
    
    for chunk in doc_chunks:
        
        all_chunks_with_metadata.append({
            "source": source_name,
            "text": chunk
        })

output_path = os.path.join(output_dir, "chunks_data.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_chunks_with_metadata, f, ensure_ascii=False, indent=4)

print(f"Total de chunks gerados: {len(all_chunks_with_metadata)}")
print(f"Dados de chunks salvos em: {output_path}")

if all_chunks_with_metadata:
    print(f"\nExemplo de metadados do primeiro chunk:")
    print(f"Fonte: {all_chunks_with_metadata[0]['source']}")
    print(f"Conteúdo: {all_chunks_with_metadata[0]['text'][:100]}...")

5 - embbending

In [ ]:

print("Carregando o modelo BGE-Large...")
model = SentenceTransformer("BAAI/bge-large-en-v1.5")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

input_path = os.path.join("processed_data", "chunks_data.json")
with open(input_path, "r", encoding="utf-8") as f:
    chunks_with_metadata = json.load(f)

all_chunks_text = [item["text"] for item in chunks_with_metadata]

print(f"Iniciando a codificação de {len(all_chunks_text)} blocos...")
embeddings = model.encode(
    all_chunks_text, 
    show_progress_bar=True, 
    batch_size=32, 
    convert_to_numpy=True
)

Carregando o modelo BGE-Large...
Iniciando a codificação de 15604 blocos...


Batches:   0%|          | 0/488 [00:00<?, ?it/s]

6 - banco vetorial FAISS

In [ ]:
dimension = embeddings.shape[1] 
index = faiss.IndexFlatL2(dimension) 
index.add(embeddings.astype('float32')) 

faiss_path = os.path.join("processed_data", "vector_index.faiss")
faiss.write_index(index, faiss_path)

print(f"\n✅ Matriz de embeddings concluída: {embeddings.shape}")
print(f"✅ Índice FAISS salvo em: {faiss_path}")


✅ Matriz de embeddings concluída: (15604, 1024)
✅ Índice FAISS salvo em: processed_data\vector_index.faiss


7 - recuperação

In [ ]:

input_dir = "processed_data"
faiss_path = os.path.join(input_dir, "vector_index.faiss")
chunks_path = os.path.join(input_dir, "chunks_data.json")

print("Carregando índice FAISS e metadados...")
if not os.path.exists(faiss_path) or not os.path.exists(chunks_path):
    print("Erro: Arquivos de dados processados não encontrados. Rode o Pipeline de Ingestão primeiro.")
else:

    index = faiss.read_index(faiss_path)
    
    with open(chunks_path, "r", encoding="utf-8") as f:
        all_chunks_with_metadata = json.load(f)
    
    model = SentenceTransformer("BAAI/bge-large-en-v1.5")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    print(f"Sistema pronto! Usando {device} para busca.")

def retrieve(query, k=5):
    instruction = "Represent this sentence for searching relevant passages: "
    
    q_emb = model.encode([instruction + query]).astype("float32")
    
    D, I = index.search(q_emb, k)
    
    retrieved_results = []
    for idx in I[0]:

        chunk_info = all_chunks_with_metadata[idx]
        retrieved_results.append({
            "text": chunk_info["text"],
            "source": chunk_info["source"]
        })
        
    return retrieved_results

pergunta = "What are the main observations of gravitational waves in neutron stars?"
contextos = retrieve(pergunta, k=3)

print(f"\n🔎 Pergunta: {pergunta}")
print(f"📚 {len(contextos)} trechos técnicos encontrados.\n")

for i, res in enumerate(contextos):
    print(f"📍 [Trecho {i+1}] - Fonte: {res['source']}")
    print(f"📄 Conteúdo: {res['text'][:400]}...") 
    print("-" * 60)

8 - minstral

In [ ]:
if 'model' in globals():
    try:
        model.to("cpu")
    except:
        pass

torch.cuda.empty_cache()
gc.collect()

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16 
)

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

print("Carregando Mistral-7B a partir do cache/hub...")

tokenizer = AutoTokenizer.from_pretrained(model_id)

model_llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    low_cpu_mem_usage=True,
    max_memory={0: "4.8GiB"}
)

print("\n✅ Mistral-7B carregado com sucesso e pronto para ler o FAISS/JSON!")

9 - função de prompt

In [ ]:
def ask(query):
    results = retrieve(query, k=4)
    
    context_text = ""
    for i, res in enumerate(results):

        context_text += f"[Reference {i+1} from {res['source']}]: {res['text']}\n\n"

    prompt = (
        f"<s>[INST] SYSTEM: You are a rigorous scientific validator. Use ONLY the provided SOURCES to answer. "
        f"If the specific details (equations, constants, or data) are not in the SOURCES, "
        f"say 'Information not available in dataset'. DO NOT use general knowledge about physics. "
        f"Be precise about Energy Conditions and Wormhole stability as described in the text.\n\n"
        f"SOURCES:\n{context_text[:2500]}\n\n"
        f"QUESTION:\n{query} [/INST]\n"
        f"Scientific Analysis based on Sources:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model_llm.device)
    input_length = inputs.input_ids.shape[1]

    output_tokens = model_llm.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.1, 
        do_sample=True,
        repetition_penalty=1.1, 
        top_p=0.9, 
        pad_token_id=tokenizer.eos_token_id
    )

    generated_tokens = output_tokens[0][input_length:]
    clean_answer = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    return clean_answer, results

10 - teste final

In [ ]:
pergunta = "how does the wormhole solution work according to equation 3.57?"

print(f"🚀 [1/2] Gerindo memória e preparando busca...")

if 'model' in globals():
    model.to("cpu")
    device_embedding = "cpu" 
else:
    device_embedding = "cuda" if torch.cuda.is_available() else "cpu"

torch.cuda.empty_cache()
gc.collect()

try:
    print(f"🧠 [2/2] Consultando Mistral-7B (4-bit) na GPU...")
    
    resposta, results = ask(pergunta)

    print("\n" + "═"*60)
    print(f"🌌 RESPOSTA CIENTÍFICA:")
    print("-" * 60)
    print(resposta)
    print("═"*60)

    print("\n📚 FONTES UTILIZADAS (Ancoragem):")
    for i, res in enumerate(results[:3]):

        print(f"   [{i+1}] Fonte: {res['source']}")
        print(f"       Trecho: \"{res['text'][:150]}...\"")
        print("-" * 30)

except torch.cuda.OutOfMemoryError:
    print("\n❌ ERRO DE MEMÓRIA: A GPU esgotou. Tente reduzir o 'k' na função retrieve.")
    torch.cuda.empty_cache()

except Exception as e:
    print(f"\n❌ Ocorreu um erro: {e}")

11 - metricas de desempenho

11.1 - indice de fidelidade semantica

A métrica de Fidelidade Semântica serve para medir o grau de honestidade intelectual do seu assistente em relação aos documentos fornecidos, funcionando como um detector de mentiras que impede a inteligência artificial de ignorar os artigos científicos para inventar respostas baseadas apenas em seu treinamento prévio

In [ ]:
query_text = pergunta 
answer_text = resposta
contexto_consolidado = " ".join([res['text'] for res in results])

print("Calculando vetores de fidelidade...")
ans_emb = model.encode([answer_text])
ctx_emb = model.encode([contexto_consolidado])

fidelidade_score = cosine_similarity(ans_emb, ctx_emb)[0][0]

print("\n" + "═"*50)
print(f"🔬 RELATÓRIO DE VERIFICAÇÃO CIENTÍFICA")
print("-" * 50)
print(f"📊 Score de Fidelidade: {fidelidade_score:.4f}")

if fidelidade_score > 0.82:
    status = "✅ EXCELENTE: Resposta totalmente ancorada nos artigos."
elif fidelidade_score > 0.65:
    status = "⚠️ ATENÇÃO: A resposta é relevante, mas pode conter inferências externas ao dataset."
else:
    status = "❌ ALERTA: Possível alucinação ou resposta baseada apenas no conhecimento prévio do modelo."

print(f"Status: {status}")
print("═"*50)

q_emb = model.encode([query_text])
relevancia_pergunta = cosine_similarity(ans_emb, q_emb)[0][0]
print(f"🎯 Relevância da Resposta para a Pergunta: {relevancia_pergunta:.4f}")

11.2 - metrica de alucinação negativa

a métrica de Alucinação Negativa, ou Gap Analysis, é projetada para identificar se o modelo está de fato extraindo informações úteis dos textos ou se está apenas realizando um "enrolation" técnico ao parafrasear a sua pergunta.

In [ ]:
ans_emb = model.encode([resposta])
query_emb = model.encode([pergunta])

sim_pergunta = cosine_similarity(ans_emb, query_emb)[0][0]

hallucination_gap = fidelidade_score - sim_pergunta

print("\n" + "═"*50)
print(f"📊 MÉTRICA DE AGREGAÇÃO TÉCNICA (Gap Analysis)")
print("-" * 50)
print(f"📈 Fidelidade (Resposta-Contexto): {fidelidade_score:.4f}")
print(f"🔄 Repetição (Resposta-Pergunta):  {sim_pergunta:.4f}")
print("-" * 30)
print(f"🎯 GAP DE CONHECIMENTO: {hallucination_gap:.4f}")

if hallucination_gap > 0.10:
    status = "✅ SEGURO: O modelo extraiu e sintetizou informações novas dos artigos."
elif hallucination_gap > 0.0:
    status = "🟡 NEUTRO: O modelo seguiu a pergunta, mas a contribuição dos artigos foi moderada."
elif hallucination_gap > -0.10:
    status = "⚠️ ALERTA: A resposta está muito presa ao texto da pergunta (Parafraseamento)."
else:
    status = "❌ CRÍTICO: Possível alucinação ou resposta genérica ignorando os fatos dos PDFs."

print(f"\nStatus: {status}")
print("═"*50)

11.3 - métrica de densidade de informação

a Densidade de Informação foca na qualidade estrutural e na sofisticação do texto produzido, sendo fundamental para evitar que o modelo entre em ciclos de repetição ou utilize uma linguagem excessivamente genérica e simplista.

In [ ]:
palavras_limpas = re.findall(r'\w+', resposta.lower())
vocabulario_unico = set(palavras_limpas)

total_palavras = len(palavras_limpas)
total_unicas = len(vocabulario_unico)
densidade = total_unicas / total_palavras if total_palavras > 0 else 0

print("\n" + "═"*50)
print(f"📊 MÉTRICA DE RIQUEZA LEXICAL")
print("-" * 50)
print(f"Total de palavras na resposta: {total_palavras}")
print(f"Vocabulário único:             {total_unicas}")
print("-" * 30)
print(f"Índice de Densidade:           {densidade:.4f}")

if densidade > 0.65:
    status = "✅ EXCELENTE: Texto denso, técnico e sem repetições desnecessárias."
elif densidade > 0.45:
    status = "🟡 NORMAL: Fluidez adequada para uma explicação científica."
else:
    status = "⚠️ REPETITIVO: O modelo pode estar 'preso' em um loop. Considere subir o 'repetition_penalty'."

print(f"\nStatus: {status}")
print("═"*50)

11.4 - métrica de precisão de recuperação(analise sobre o BGE-large)

In [ ]:
contextos_texto = [res['text'] for res in results]

query_embedding = model.encode([pergunta])

context_embeddings = model.encode(contextos_texto)

similaridades = cosine_similarity(query_embedding, context_embeddings)[0]

mean_similarity = np.mean(similaridades)
max_similarity = np.max(similaridades)

print("\n" + "═"*50)
print(f"📊 PERFORMANCE DO RETRIEVER (BGE-LARGE)")
print("-" * 50)
print(f"Similaridade Máxima (Top 1):  {max_similarity:.4f}")
print(f"Similaridade Média (Top {len(similaridades)}):   {mean_similarity:.4f}")
print("-" * 30)

if max_similarity > 0.75:
    status = "✅ ALTA RELEVÂNCIA: O BGE encontrou trechos muito específicos nos artigos."
elif max_similarity > 0.55:
    status = "🟡 RELEVÂNCIA MÉDIA: O conteúdo é correlato, mas pode ser genérico."
else:
    status = "❌ BAIXA RELEVÂNCIA: O banco de 1000 PDFs pode não conter a resposta exata."

print(f"Status: {status}")
print("═"*50)